# Investigation on Titanic

The Titanic dataset was investigateed in this file. By exploring the dataset, we've found out that among all the passengers, the **cabinet class, sex and age** are effecting the survival rate. 

## Questions about the dataset
By looking at the dataset, the most important thing from this dataset is to find out which people survived, and what are the common features do the survived people share. 

According to the head of the table, there are several interesting parameters which might contribute to the survival:
- **1. Pclass**: Do higher class passengers have higher survival rate? 
- **2. Sex**: Do Ladies have higher survival rate? 
- **3. Age**: Do elder people have higher survival rate or lower?
- **4. Age, Sex**: Do elder ladies and young children have higher survival rate?


##  Load Data from CSVs
In order to explore which parameters contribute to the survival, let's first have a look at the data we got.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%pylab inline

In [ ]:
df = pd.read_csv("titanic_data.csv")
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Pclass'].unique()

In [ ]:
df['Sex'].unique()

In [ ]:
df['Ticket'].unique()

In [13]:
df['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104', 'C111',
       'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30', 'D20',
       'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22', 'C86',
       'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20', 'D19',
       'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126', 'B71',
       'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64', 'E24',

There are quite a few different cabins, but we also noticed that they might be categorized as the first letter: A~T

In [17]:
df[df['Cabin'] == 'T']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
339,340,0,1,"Blackwell, Mr. Stephen Weart",male,45.0,0,0,113784,35.5,T,S


##   (Question 1) Pclass : Do higher class passengers have higher survival rate? ##
Cabinet class is first studied here. A plot is shown for each class to visualize the survival data. 

In [ ]:
pclass = df.groupby(["Pclass", "Survived"] )
# print df.groupby(["Pclass","Survived"]).groups
print "Total passenger number: ", len(df)

print len(pclass["Pclass"])
three_n = len(pclass.groups[(3,0)])
three_sv = len(pclass.groups[(3,1)])
two_n = len(pclass.groups[(2,0)])
two_sv = len(pclass.groups[(2,1)])
one_n = len(pclass.groups[(1,0)])
one_sv = len(pclass.groups[(1,1)])

th_sv = three_sv/float(three_sv+ three_n)
t_sv = two_sv/float(two_sv+ two_n)
o_sv = one_sv/float(one_sv+ one_n)

print "Survival rate for Pclass 3: ", three_sv/float(three_sv+ three_n)
print "Survival rate for Pclass 2: ", two_sv/float(two_sv+ two_n)
print "Survival rate for Pclass 1: ", one_sv/float(one_sv+ one_n)

plt.bar([1,2,3], [ o_sv, t_sv, th_sv], width = 0.5)
plt.xticks([1,2,3], ["Class 1", "Class 2", "Class 3"])

**Pclass contributes to the survival rate.**

Among the three different classes, Pclass 1 has the highest survival rate of 63.0%, Pclass 2 has higher rate than Pclass 3. In this case we can conclude that Pclass is an important parameter.

## (Question 2) Sex: Do Ladies have higher survival rate?  ##



In [ ]:
size = []
size.append(len(df[df["Sex"] == "male"]))
size.append(len(df[df["Sex"] == "female"]))
print len(df[df["Sex"] == "male"])
print len(df[df["Sex"] == "female"])
print size

labels = ['Male', 'Female']
colors = ['red', 'blue']

plt.pie(size, labels=labels, colors=colors, autopct='%1.1f%%')

The pie chart about the Sex groups shows that the male and female population are quite close.

In [ ]:
sx_m = df['Survived'][df['Sex']=='male'].value_counts()
sx_f = df['Survived'][df['Sex']=='female'].value_counts()

print "Male survival: \n", sx_m
print '\n'
print "Female survival: \n", sx_f

fm = 100*sx_f[1]/float(sx_f.sum())
mm = 100*sx_m[1]/float(sx_m.sum())
a = 'Female\n' + str(round(fm, 0)) + '% Survived'
b = 'Male\n' + str(round(mm, 0)) + '% Survived'
                   
plt.ylabel('Number')
plt.xticks((0.125,1.125),(a,b))
survived_num = plt.bar((0, 1), [sx_f[1],sx_m[1]], width = 0.125, color = 'red')
total_num = plt.bar((0.125,1.125),[sx_f.sum(),sx_m.sum()], width = 0.125, color = 'blue' )
plt.legend([total_num,survived_num],['Total number','Suvived number'],loc = 'upper left')

According to the survival rate simply grouped by **Sex**, female has much higher survival rate than Male. 

##  Clean the data ## 

However, by looking at the table it is found that there are 177 missing values for the Age, which shall not be ignored. Considering that "Adult" group is the major group among the "Age", so the missing values were valued as "Adult".

Thus here the provided ages can be grouped as **"Child", "Youth", "Adult" and "Elder"**

In [ ]:
print "There are {} empty value for Age group.".format(df["Age"].isnull().sum())
print "Total population is: ", len(df)

In [ ]:
def age_range(age):
    if age >=0 and age <12:
        return "Child"
    elif age < 18:
        return "Youth"
    elif age < 60:
        return "Adult"
    elif age > 60:
        return "Elder"
    else:
        return "Adult"

df["Age"] = df["Age"].apply(age_range)

##  (Question 3)  Age: Do elder people have higher survival rate or lower?  ##

In [ ]:
#Group the "Sex" and "Age" group
age = df.groupby(["Age"], as_index = False)

#Count the total amount of people in each group in "age_sum"
age_sum = age['Survived'].count()

#Count the survived people in each group in "age_sv"
age_sv = age.aggregate({'Survived': np.sum})

#Calculate the survival rate for each group
age_rate = age_sv["Survived"].div(age_sum["Survived"], axis='index') 

print "Total amount: \n", age_sum
print "Survived: \n",age_sv
print "Survival rate: "
print age_rate
#age_rate.plot(kind = "bar")

plt.xticks(range(4),["Adult", "Child", "Elder", "Youth"])
plt.xlim([-1,4.5])
plt.ylim([-0.1,1])
plt.ylabel('Survival rate')
plt.bar(range(4), age_rate, width = 0.25)
plt.title("The survival rate of different Age groups")

According to the survival rate simply grouped by **Age**, though the elder are lower than the others, but even NaN group has average survival rate. It's hard to get any conclusion from this chart simply by grouping them into Age groups. 

##  (Question 4) Sex, Age: Do Ladies and young children have higher survival rate?  ##

More parameters are studied. Another intuition is the **Gender** and **Age**. 

In [ ]:
#Group the "Sex" and "Age" group
sxage = df.groupby(["Sex", "Age"], as_index = False)

#Count the total amount of people in each group in "sxage_sum"
sxage_sum = sxage['Survived'].count()

#Count the survived people in each group in "sxage_sv"
sxage_sv = sxage.aggregate({'Survived': np.sum})

print "Total amount: \n", sxage_sum
print "\n"
print "Survived: \n",sxage_sv

In [ ]:
#Calculate the survival rate for each group
sxage_rate = sxage_sv["Survived"].div(sxage_sum["Survived"], axis='index') 
print sxage_rate

In [ ]:
#Visuallize the survival rate for each group
#plt.bar(range(4), sxage_rate[4:8], width = 0.35, position=1)
#plt.bar(range(4), sxage_rate[0:4], width = 0.35, position=0)
print sxage_rate[4:8]

x = [1, 2,3,4]
xx = [0.6,1.6,2.6,3.6]
ax = plt.subplot(111)
ax.bar(xx,sxage_rate[4:8],width=0.35,color='r',align='center')

ax.bar(x,sxage_rate[0:4],width=0.35,color='b',align='center')
ax.set_xticklabels(("","Adult","","Child","", "Elder", "","Youth"))
ax.legend(('Male', 'Female'))


It is shown in this bar chart that **Female** has much generally higher survival rate than male. **Age** is also effecting the rate, among this **Femal Elder** (100%) and **Femal Youth** (82.6%) has the highest survival rate, while among the male list, **Male Child** (55.6%) has much higher survival rate than the others. 

However, due to that there're some people has age data missing **(NaN)**, we are not 100% sure that "Female Elder" all survived. So more parameters need to be explored.

## Conclusion ##
This chart shows the combination of "Pclass", "Sex" and "Age". 
1. Pclass has effect on the survival rate. 
    It is clearly shown that Pclass 3 has much lower survival rate, while the difference between Pclass 1 and 2 are not clear. 
2. Age and Sex both has effect on the survival rate. 
    Female has higher survival rate than male, and Female Elder has the highest among all. Male Adult and Elder has very low rate. However, Male Youth has high survival rate in Pclass 1 and 2 (both 100%), while 33% at Pclass 3. 
3. So far it is clear that there's correlation between Pclass, Sex and Age, but it's not causation relationship. Some Machine Learning calculation is required to oder to find out which parameter contributes more.  


In [ ]:
# import the D3 module
from IPython.core.display import HTML
import d3_lib